In [1]:
from config import *

import os
import re
import pandas as pd

# r: the reference person
# s: spouse
# h: househould

h_waveless_col = ['hidpn']
r_waveless_col = ['gender', 'edyrs', 'bplace']
waveless_col = [f'h{col}' for col in h_waveless_col] + \
    [f'ra{col}' for col in r_waveless_col]
h_wave_col = ['child']
r_wave_col = [
    # demographics
    'proxy', 'agey_m', 'cenreg', 'cendiv', 'mstat',
    'momliv', 'dadliv', 'livpar', 'momage', 'dadage',
    'livbro', 'livsis', 'livsib',

    # employment history
    'hlthlm', 'lbrf',

    # health
    'shlt', 'bmi', 'height', 'weight',  # self-report
    'hosp', 'hsptim', 'hspnit',  # hospital
    'nrshom', 'nhmliv', 'nrstim', 'nrsnit',  # nursing house
    'doctor', 'doctim',  # doctor visit
    'depres', 'effort', 'sleepr', 'whappy', 'flone', 'fsad', 'going', 'enlife',  # mental health
    # diagnosed health problems
    'hibpe', 'diabe', 'cancre', 'lunge', 'hearte', 'stroke', 'arthre',
    'vgactx', 'drink', 'drinkd', 'smokev', 'smoken',  # health behaviours

    # cognition
    'slfmem', 'mo', 'dy', 'yr', 'dw',

    # Activities of daily living (ADLs): Any difficulty
    'walkra', 'dressa', 'batha', 'eata', 'beda', 'toilta',
    # TODO: adlwa = sum(dressa, batha, eata)
    # adl5a = sum(walkra, dressa, batha, eata, beda)
    # adl6a = sum(walkra, dressa, batha, eata, beda, toilta)
    'adl5a',  # adlwa

    # Instrumental activities of daily living (IADLs): Any difficulty
    'mapa', 'phonea', 'moneya', 'medsa', 'shopa', 'mealsa',
    # TODO: iadl3a = sum(phonea, moneya, medsa), referred as "iadla" in appendix
    # iadl5a = sum(phonea, moneya, medsa, shopa, mealsa), referred as "iadlza" in appendix
    'iadl5a',

    # Other Functional Limitations: Any difficulty
    'walksa', 'walk1a', 'sita', 'chaira', 'climsa', 'clim1a',
    'stoopa', 'lifta', 'dimea', 'armsa', 'pusha',
    # mobila = sum(walksa, walk1a, walkra, climsa, clim1a)
    # lgmusa = sum(sita, chaira, stoopa, pusha)
    # grossa = sum(walk1a, walkra, clim1a, beda, batha)
    # finea = sum(dimea, eata, dressa)
    'mobila',  'lgmusa', 'grossa', 'finea',

    # NOTE: variables that are meant to encode the dependent variable should not be utilized as features.
    'imrc', 'dlrc', 'ser7', 'bwc20', 'prmem',
    # TODO prfin = NA011(2012); OA011(2014)
    # TODO demscr (self-respondent) = sum(imrc, dlrc, ser7, bwc20)
    # TODO demscr (proxy-respondent) = sum(prmem, iadl5a, prfin)
    # tr20 = sum(imrc, dlrc)
    'tr20',
]

# Define wave and year based on wave number
# For wave 11 (2012), it's used for training and internal validation
# For wave 12 (2014), it's used for external validation
wave = 12
year = 2 * (wave - 5)

# Read the main dataset
columns = [col for col in waveless_col] + \
    [f'h{wave}{col}' for col in h_wave_col] + \
    [f'r{wave}{col}' for col in r_wave_col]
chunks_main: pd.DataFrame = pd.read_stata(
    FILENAME, columns=columns, iterator=False)

In [2]:
# Map the corresponding prefin column based on the year
col_prfin = {12: 'NA011', 14: 'OA011'}
prfin_o = col_prfin[year]
prfin = f'r{wave}prfin'
# Read the prefin column dataset and perform preprocessing
chunks_prfin = pd.read_sas(os.path.join(DIR_RO, f'h{year}a_r.sas7bdat'))
chunks_prfin['hhidpn'] = (chunks_prfin['HHID'] +
                          chunks_prfin['PN']).astype(int)
# chunks_prfin[prfin] = chunks_prfin[prfin_o] - 1
chunks_prfin[prfin] = pd.cut(chunks_prfin[prfin_o], bins=[0, 1, 2, 3], labels=[
                             '1.none', '2.some', '3.prevented'], ordered=True)
chunks_prfin = chunks_prfin[['hhidpn', prfin]]
# Merge prefin column with the main dataset
chunks: pd.DataFrame = pd.merge(
    chunks_main, chunks_prfin, left_on='hhidpn', right_on='hhidpn', how='left')
chunks.drop([f'h{col}' for col in h_waveless_col], axis=1, inplace=True)

In [3]:
for col in ['ragender', 'raedyrs']:
    wave_col = col
    chunks.loc[:, wave_col] = chunks[wave_col].cat.codes.astype('float')
    chunks.loc[chunks[wave_col] == -1, wave_col] = float('nan')

# ragender: ['1.male' < '2.female'] => float
# raedyrs: ['0.none' < 1.0 < 2.0 < 3.0 ... 14.0 < 15.0 < 16.0 < '17.17+ yrs'] => float + one more variable 'redleq17' to indicate 17 or more
# raedstg: ['1.0-7' < '2.8-11' < '3.12' < '4.13+']
# rabplace: [
#     '1.new england', '2.mid atlantic', '3.en central', '4.wn central', '5.s atlantic',
#     '6.es central',  '7.ws central',   '8.mountain',   '9.pacific',    '10.us/na division',
#     '11.not us/inc us terr', 'nan']
# child: float

for col in [
    'proxy', 'momliv', 'dadliv', 'hlthlm', 'hosp', 'nrshom', 'nhmliv', 'doctor',
    'depres', 'sleepr', 'whappy', 'flone', 'fsad', 'going', 'enlife',
    'drink', 'drinkd', 'smokev', 'smoken', 'mo', 'dy', 'yr', 'dw',
    'walkra', 'dressa', 'batha', 'eata', 'beda', 'toilta',
    'mapa', 'phonea', 'moneya', 'medsa', 'shopa', 'mealsa',
    'walksa', 'walk1a', 'sita', 'chaira', 'climsa', 'clim1a', 'stoopa', 'lifta', 'dimea',
    'armsa', 'pusha', 'bwc20', 'prmem', 'prfin',
]:
    wave_col = f'r{wave}{col}'
    chunks.loc[:, wave_col] = chunks[wave_col].cat.codes.astype('float')
    chunks.loc[chunks[wave_col] == -1, wave_col] = float('nan')

# chunks.loc[chunks['raedyrs'] <= 17, 'redleq17'] = 0.0
# chunks.loc[chunks['raedyrs'] == 17, 'redleq17'] = 1.0
chunks.loc[~chunks['raedyrs'].isna(), 'raedstg'] = pd.cut(
    chunks.loc[~chunks['raedyrs'].isna(), 'raedyrs'],
    bins=[-1, 7, 11, 12, chunks['raedyrs'].max()],
    labels=['1.0-7', '2.8-11', '3.12', '4.13+'])

# proxy: ['0.not proxy' < '1.proxy'] => float
# agey_m: float
# cenreg: ['1.northeast' < '2.midwest' < '3.south' < '4.west' < '5.other']
# cendiv: [
#     '1.new england', '2.mid atlantic', '3.en central', '4.wn central', '5.s atlantic',
#     '6.es central',  '7.ws central',   '8.mountain',   '9.pacific',    '11.not us/inc us terr',
#     'nan']
# mstat: ['1.married' < '2.married,spouse absent' < '3.partnered' < '4.separated' < '5.divorced' < '7.widowed' < '8.never married']
# momliv: ['0.no' < '1.yes'] => float

# dadliv: ['0.no' < '1.yes'] => float
# livpar, momage, dadage, livbro, livsis, livsib: float
# hlthlm: ['0.no' < '1.yes'] => float
# lbrf: ['1.works ft' < '2.works pt' < '3.unemployed' < '4.partly retired' < '5.retired' < '6.disabled' < '7.not in lbrf']
# shlt: ['1.excellent' < '2.very good' < '3.good' < '4.fair' < '5.poor']

# bmi, height, weight: float
# hosp: ['0.no' < '1.yes'] => float
# hsptim, hspnit: float
# nrshom, nhmliv: ['0.no' < '1.yes'] => float
# nrstim, nrsnit: float

# doctor: ['0.no' < '1.yes'] => float
# doctim: float
# depres, effort, sleepr, whappy, flone, fsad, going, enlife: ['0.no' < '1.yes'] => float

# hibpe, diabe, cancre, lunge, hearte, stroke, arthre: ['0.no' < '1.yes'] => float
# vgactx: ['1.every day' < '2.>1 per week' < '3.1 per week' < '4.l-3 per mon' < '5.never']
# drink: ['0.no' < '1.yes']
# drinkd: ['0.0 or doesnt drink' < 1.0 < 2.0 < 3.0 < 4.0 < 5.0 < 6.0 < 7.0] => float

# smokev, smoken: ['0.no' < '1.yes'] => float
# slfmem: ['1.excellent' < '2.very good' < '3.good' < '4.fair' < '5.poor']
# mo, dy, yr, dw: ['0.incorrect' < '1.correct'] => float
# walkra, dressa, batha: ['0.no' < '1.yes'] => float

# eata, beda, toilta: ['0.no' < '1.yes'] => float
# adl5a: float
# mapa, phonea, moneya, medsa, shopa, mealsa: ['0.no' < '1.yes'] => float

# iadl5a: float
# walksa, walk1a, sita, chaira, climsa, clim1a, stoopa, lifta, dimea: ['0.no' < '1.yes'] => float

# armsa, pusha: ['0.no' < '1.yes'] => float
# mobila, lgmusa, grossa, finea, imrc, dlrc, ser7: float
# bwc20: ['0.incorrect' < '1.correct, 2nd try' < '2.correct, 1st try'] => float

# prmem: ['1.excellent' < '2.very good' < '3.good' < '4.fair' < '5.poor'] => float
# tr20: float
# prfin: ['1.none' < '2.some' < '3.prevented']

chunks.drop(['raedyrs'], axis=1, inplace=True)

In [4]:
# adlwa
adlawa_lst = [f'r{wave}{col}' for col in ['dressa', 'batha', 'eata']]
chunks[f'r{wave}adlwa'] = chunks[adlawa_lst].sum(
    axis=1, min_count=len(adlawa_lst))

# iadl3a
iadlala_lst = [f'r{wave}{col}' for col in ['phonea', 'moneya', 'medsa']]
chunks[f'r{wave}iadla'] = chunks[iadlala_lst].sum(
    axis=1, min_count=len(iadlala_lst))

In [5]:
# demscr
# recode dependent variable
proxy = f'r{wave}proxy'
demscr = f'r{wave}demscr'
demcls = f'r{wave}demcls'

resp_self = chunks[chunks[proxy] == 0].copy()
wave_self_dem_cols = [f'r{wave}{col}' for col in SELF_DEM_COLS]
self_mask = (chunks[proxy] == 0)
chunks.loc[self_mask, demscr] = chunks.loc[self_mask][wave_self_dem_cols].sum(
    axis=1, min_count=len(wave_self_dem_cols))
chunks.loc[(self_mask) & (chunks[demscr] <= 6), demcls] = 1.0
chunks.loc[(self_mask) & (chunks[demscr] > 6), demcls] = 0.0

resp_proxy = chunks[chunks[proxy] == 1].copy()
wave_proxy_dem_cols = [f'r{wave}{col}' for col in PROXY_DEM_COLS]
proxy_mask = (chunks[proxy] == 1)
chunks.loc[proxy_mask, demscr] = chunks.loc[proxy_mask][wave_proxy_dem_cols].sum(
    axis=1, min_count=len(wave_proxy_dem_cols))
chunks.loc[(proxy_mask) & (chunks[demscr] >= 6), demcls] = 1.0
chunks.loc[(proxy_mask) & (chunks[demscr] < 6), demcls] = 0.0

chunks: pd.DataFrame
# missing values may occur in proxy survey
chunks.dropna(subset=demscr, inplace=True)
chunks.drop([demscr], axis=1, inplace=True)

In [6]:
chunks.rename(columns={col: re.sub(
    f'^[r|h](a|{wave})', "", col) for col in chunks.columns}, inplace=True)
new_column_order = [
    'gender', 'edstg', 'bplace', 'child', 'proxy', 'agey_m', 'cenreg', 'cendiv',
    'mstat', 'momliv', 'dadliv', 'livpar', 'momage', 'dadage', 'livbro',
    'livsis', 'livsib', 'hlthlm', 'lbrf', 'shlt', 'bmi', 'height', 'weight',
    'hosp', 'hsptim', 'hspnit', 'nrshom', 'nhmliv', 'nrstim', 'nrsnit',
    'doctor', 'doctim', 'depres', 'effort', 'sleepr', 'whappy', 'flone',
    'fsad', 'going', 'enlife', 'hibpe', 'diabe', 'cancre', 'lunge',
    'hearte', 'stroke', 'arthre', 'vgactx', 'drink', 'drinkd', 'smokev',
    'smoken', 'slfmem', 'mo', 'dy', 'yr', 'dw', 'walkra', 'dressa', 'batha',
    'eata', 'beda', 'toilta', 'adlwa', 'adl5a', 'mapa', 'phonea', 'moneya', 'medsa',
    'shopa', 'mealsa', 'iadla', 'iadl5a', 'walksa', 'walk1a', 'sita', 'chaira',
    'climsa', 'clim1a', 'stoopa', 'lifta', 'dimea', 'armsa', 'pusha',
    'mobila', 'lgmusa', 'grossa', 'finea', 'imrc', 'dlrc', 'ser7', 'bwc20',
    'prmem', 'tr20', 'prfin', 'demcls']
chunks = chunks.reindex(columns=new_column_order)
chunks.reset_index(drop=True, inplace=True)
chunks.to_stata(os.path.join(
    DIR_RW, f'w{wave}_subsets.dta'), write_index=False)

In [7]:
chunks_desp = chunks.describe().loc[['count', 'mean', 'std', 'min', 'max']].T
chunks_desp['count'] = chunks_desp['count'].astype(int)
chunks_desp = chunks_desp.round(3)

chunks_desp

count    mean     std   min    max
gender  18738   0.589   0.492   0.0    1.0
child   18461   3.145   2.055   0.0   11.0
proxy   18738   0.056   0.229   0.0    1.0
agey_m  18738  67.899  11.264  18.0  104.0
momliv  18512   0.213   0.409   0.0    1.0
...       ...     ...     ...   ...    ...
bwc20   17698   1.851   0.522   0.0    2.0
prmem    1040   2.505   1.371   0.0    4.0
tr20    17698   9.645   3.545   0.0   20.0
prfin    1040   1.109   0.959   0.0    2.0
demcls  18738   0.074   0.261   0.0    1.0

[79 rows x 5 columns]